<a id=top></a>

# **目次**

<b>
    <details>
        <summary>
            <a href="#modules", style="font-size: xx-large">1. モジュールインポート</a>
            <ul>※サードパーティライブラリ>>>自作モジュール>>>（ここまで本ipynb外）>>>自作関数（本ipynb内）</ul>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#getdata", style="font-size: xx-large">2. データのゲット</a>
        </summary>
    </details>
    <details>
        <summary>
            <a href="#melttargz", style="font-size: xx-large">3. targz展開</a>
        </summary>
        <table></table>
    </details>
</b>


---


<a id=modules></a>

## **1. モジュールインポート**


In [1]:
import pandas as pd
import numpy as np
import time
import json
from glob import glob
import requests
from bs4 import BeautifulSoup
import urllib.request
import tarfile


In [10]:
global data_dir, excute_count
data_dir = '../../data/original/internal/'
excute_count = 0

In [ ]:
# データ取得
def get_data(master_name, coding='utf-8'):
    html_file = f'{data_dir}bulk_html/{master_name}.html'
    output_dir = f'{data_dir}bulk_targz/{master_name}/'
    
    with open(html_file, 'r', encoding=coding) as f:
        html_text = f.read()
    
    soup = BeautifulSoup(html_text, 'html.parser')
    href_list = list(set(href for href in [a.get('href') for a in soup.select('div a')]\
                                          if master_name in href))
    for href in href_list:
        urllib.request.urlretrieve(href, filename=output_dir+href.split('/')[-1])
        # print(f'Downloaded: {href.split("/")[-1]}')
        time.sleep(3)
    print(master_name, 'got!')


In [16]:
# データ解凍
def extract_data(master_name):
    needed_file_dict = {
                        'JPWRP': [# 登録マスタ
                                  'upd_mgt_info_p.tsv', 
                                  'upd_right_person_art_p.tsv', 
                                 ], 
                        'JPWIP': [# IPCマスタ
                                 'upd_dsptch_fin_ipc.tsv'
                                 ], 
                        'JPWAP': [# 出願マスタ
                                 'upd_pmac_g_app_case.tsv', 
                                 'upd_sinseinin.tsv'
                                 ]
                        }
    targz_list = glob(f'{data_dir}bulk_targz/{master_name}/*')
    for targz in targz_list:
        file_date = targz.split(f'{master_name}_')[-1].split('.')[0]
        
        with tarfile.open(name=targz, mode='r:gz') as tf:
            member_list = tf.getmembers()
            for member in member_list:
                file_name = member.name.split('/')[-1]
                if file_name in needed_file_dict[master_name]:
                    needed_dir = file_name.split('.')[0]
                    
                    # 誰に何と言われようと、バージョンに依存するメソッドはくそです。
                    tf.extract(member, 
                               path=f'{data_dir}bulk/{master_name}/{needed_dir}/{file_date}.tsv')
                    # print(f'Extracted: {folder_name}/{file_name}')
            tf.close()
    print(master_name, 'melted!')


In [11]:
import tarfile
master_name = 'JPWRP'
targz_list = glob(f'{data_dir}bulk_targz/{master_name}/*')
for targz in targz_list:
    # print(targz)
    with tarfile.open(name=targz, mode='r:gz') as tar:
        for i in tar.getmembers():
            if i.name[0] == '/' or i.name[0:2] == '..':
                print('外のディレクトリに展開されるかも。')
                exit()
            print(i.name.split('.'))

TypeError: 'regex' is an invalid keyword argument for split()

In [17]:
for mn in ['JPWRP', 'JPWIP', 'JPWAP']:
    # if excute_count == 0: get_data(mn)
    excute_count += 1
    extract_data(mn)
print('completed!')


JPWRP melted!
JPWIP melted!


KeyboardInterrupt: 